In [27]:
import ee
import folium
import numpy as np
import pandas as pd
import proplot as plot
import matplotlib.pyplot as plt
import datetime
from pandas.plotting import register_matplotlib_converters

service_account = 'nattawat@geeproject-313808.iam.gserviceaccount.com'
credentials = ee.ServiceAccountCredentials(service_account, 'key.json')
ee.Initialize(credentials)

In [28]:
from geojson import Polygon


coor  = [[101.2142944, 15.0590284],[101.2407303, 15.0565419],[101.2422752, 15.0787537],[101.2151527, 15.0805769],[101.2142944, 15.0590284]]
polygon = ee.Geometry.Polygon(coor)

mask = 0

F_date = (input("Frist Date : ")) #2018-01-01
L_date = (input("Last Date : ")) #2018-01-13 

Poly = (input("Polygon : "))
print(f'Frist Date is : {F_date} \nLast date is : {L_date}\nPolygon : {Poly}')



##############################################
def Download_IMGAE(F_date,polygon):

    def maskSen2_clouds(image):
        qa = image.select('QA60')

    # Bits 10 and 11 are clouds and cirrus, respectively.
        cloudBitMask = 1 << 10
        cirrusBitMask = 1 << 11

    # Both flags should be set to zero, indicating clear conditions.
        mask = qa.bitwiseAnd(cloudBitMask).eq(0)
        mask = mask.bitwiseAnd(cirrusBitMask).eq(0)

        return image.updateMask(mask).divide(10000)

    ###################################
    SATELLITE_ST = 'COPERNICUS/S2'
    first_date = F_date
    last_date = L_date

    print('Polygon GeoJSON: ', polygon.toGeoJSONString()) # Print the geometry as a GeoJSON string.
    print('Geometry type: ', polygon.type().getInfo()) # Print the GeoJSON 'type'

    # Call collection of satellite images.
    collection = (ee.ImageCollection(SATELLITE_ST)
              # Select the Red, Green and Blue image bands, as well as the cloud masking layer.
                 .select(['B8','B4', 'B3', 'B2','QA60'])
              # Filter for images within a given date range.
                 .filter(ee.Filter.date(first_date,last_date))
              # Filter for images that overlap with the assigned geometry.
                .filterBounds(polygon)
              # Filter for images that have less then 20% cloud coverage.
                .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)).map(maskSen2_clouds))
              
    image = collection.sort('system:index', opt_ascending=False).mosaic()
    image_RGB = image.visualize( bands=['B4', 'B3', 'B2'],#True color
                                 min=[0.0, 0.0, 0.0],
                                 max=[0.3,0.3, 0.3] )

    image_NRG = image.visualize( bands=['B8', 'B4', 'B3'], #False color
                                 min=[0.0, 0.0, 0.0],
                                 max=[0.3,0.3, 0.3] )        
             
    task_config = {
        'fileFormat': 'GeoTIFF',
        'region': polygon.coordinates().getInfo(),
        'folder': 'Example_Folder_Name',
        'scale': 10,
        'crs': 'EPSG:4326',
        'description': 'Example_File_Name'}

    link_True = image_RGB.getDownloadUrl(task_config)
    link_False = image_NRG.getDownloadUrl(task_config)

    # display('Coppy URL to download True : ', link_True)
    # display('Coppy URL to download False : ', link_False)

    return display('Coppy URL to download True : ', link_True),display('Coppy URL to download False : ', link_False)

Download_IMGAE(F_date,polygon)

Frist Date is : 2018-01-01 
Last date is : 2018-01-13
Polygon : 5
Polygon GeoJSON:  {"type": "Polygon", "coordinates": [[[101.2142944, 15.0590284], [101.2407303, 15.0565419], [101.2422752, 15.0787537], [101.2151527, 15.0805769], [101.2142944, 15.0590284]]], "evenOdd": true}
Geometry type:  Polygon


'Coppy URL to download True : '

'https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/b75254a0c785c9e344bceef17c38e088-039cc15c0dcc8fa73562186544b12920:getPixels'

'Coppy URL to download False : '

'https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/db536bc2c6861037dd54748f5b95789b-d504ecf9da6237685f457b3ac92e9e1a:getPixels'

(None, None)